# Apply document classifier

This script applies the trained classifier to scraped documents and narrows to mostly likely doi document for each district.

In [4]:
import pandas as pd
import os
import spacy
from spacy.util import minibatch, compounding
from start import data_path
import numpy as np

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


## Import texts

In [31]:
texts_scraped = pd.read_csv(os.path.join(data_path, 'text_scraped.csv'))
texts = texts_scraped['text']
texts.head()

0    0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...
1    Carlisle Independent School District District ...
2    Lockney ISD District of Innovation Plan House ...
3    Adopted AISD Local Innovation Plan 02-16-17 co...
4    March 2017 through March 2022 Paris ISD Distri...
Name: text, dtype: object

## Import and apply model

In [32]:
# Load model
model_dir = os.path.join(os.getcwd(),  '..', '..', 'data', 'plans', 'document_classifier_n300')
# apply the saved model
print("Loading from", model_dir)
nlp = spacy.load(model_dir)
categories = []
for text in texts:
    doc = nlp(text)
    categories.append(doc.cats)

Loading from /Users/kylieleblancKylie/domino/dofis/src/data_from_plans/../../data/plans/document_classifier_n300


In [33]:
p_innovation = [x['INNOVATION'] for x in categories]

In [34]:
texts_scraped_p = texts_scraped
texts_scraped_p['p_innovation'] = p_innovation
texts_scraped_p.tail()

,Unnamed: 0,Unnamed: 0.1,title,level,type,link,text,p_innovation
3990,3990,396,Petersburg ISD,html,html,http://www.petersburgisd.net/DOI,District of Innovation - Petersburg ISD PISD S...,0.002487
3991,3991,397,Three Way ISD,html,html,https://www.twisd.us/144516_2,Three Way Independent School District - Three ...,0.012157
3992,3992,398,Sinton ISD,html,html,http://www.sintonisd.net/wp/index.php/district...,District of Innovation (DOI) Latest: Teacher o...,0.000078
3993,3993,399,Athens ISD,html,html,http://athensisd.net/cms/One.aspx?portalId=841...,District of Innovation Plan - Athens Independe...,0.000045
3994,3994,400,Spring ISD,html,html,http://www.springisd.org/doi,District of Innovation / Homepage Skip to Main...,0.000269


In [35]:
texts_scraped_p.head()

,Unnamed: 0,Unnamed: 0.1,title,level,type,link,text,p_innovation
0,0,0,Abilene ISD,First,pdf,https://www.abileneisd.org/wp-content/uploads/...,0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...,0.999955
1,1,1,Carlisle ISD,First,pdf,https://s3.amazonaws.com/scschoolfiles/83/carl...,Carlisle Independent School District District ...,0.999955
2,2,2,Lockney ISD,First,pdf,https://s3.amazonaws.com/scschoolfiles/777/doi...,Lockney ISD District of Innovation Plan House ...,0.999753
3,3,3,Arlington ISD,First,pdf,http://w4.aisd.net/pdf/District-of-Innovation-...,Adopted AISD Local Innovation Plan 02-16-17 co...,0.999955
4,4,4,Paris ISD,First,pdf,https://s3.amazonaws.com/scschoolfiles/167/doi...,March 2017 through March 2022 Paris ISD Distri...,0.999955


In [36]:
texts_scraped_p.to_csv(os.path.join(data_path, 'text_scraped_p.csv'))

## Descriptive statistics

In [6]:
texts_scraped_p = pd.read_csv(os.path.join(data_path, 'text_scraped_p.csv'))

In [7]:
print("Predicted probability of DOI plan for first level documents:")
texts_scraped_p[['p_innovation']][texts_scraped_p.level == 'First'].mean()

Predicted probability of DOI plan for first level documents:


p_innovation    0.633631
dtype: float64

In [8]:
print("Predicted probability of DOI plan for second level and HTML documents:")
texts_scraped_p[['p_innovation']][texts_scraped_p.level != 'First'].mean()

Predicted probability of DOI plan for second level and HTML documents:


p_innovation    0.115193
dtype: float64

# Narrow Documents

Narrow documents to most likely DOI plan candidates. Only keep text if classified as DIP.

In [9]:
text_narrowed = texts_scraped_p.sort_values(['title', 'p_innovation'], ascending=False).drop_duplicates(subset=["title"], keep="first")
text_narrowed['text'] = np.where((text_narrowed.p_innovation < 0.5), 'Not a plan', text_narrowed.text)

In [10]:
text_narrowed.to_csv(os.path.join(data_path, 'text_narrowed.csv'))

In [11]:
# Make first 30 documents easy to read.
for i, v in enumerate(text_narrowed.loc[:,:]['text'][0:30]):
    print('\n')
    print('-'*30)
    print('DISTRICT {}'.format(i))
    print('-'*30)
    print('\n')
    print(v[0:4000])



------------------------------
DISTRICT 0
------------------------------


LTISD Plan of Innovation LAKE TRAVIS INDEPENDENT SCHOOL DISTRICT -Effective Dec. 13, 2016 – Dec. 13, 2021 Frequently Asked Questions Nhat is a District of Innovation? Nhat are the benefits to Lake Travis ISD becoming a District of Innovation? Nhat are the proposed areas of innovation for Lake Travis ISD? Nhat areas is Lake Travis ISD prohibited from seeking exemption? What is a District of Innovation? The District of Innovation concept was passed by the 84th Legislative Session in House Bill 1842 and has been codified in the Texas Education Code Chapter 12a. A District of Innovation designation gives traditional school districts most of the flexibility currently available to Texas’ open-enrollment charter schools. It gives a district more local control and increases a district’s ability to decide what best suits the district’s needs. Under HB 1842, districts may identify certain requirements imposed by the Tex

## Narrowed Descriptives

In [12]:
test = pd.read_csv(os.path.join(data_path, 'text_narrowed.csv'))
test = test[test.text != '']

In [13]:
print('Average probability of DOI classification: ', test.p_innovation.mean())
print('SD of DOI classification: ',test.p_innovation.std())
print('Number of documents w/<.2 probability of DOI classification: ', len(test[test.p_innovation < .2]))

Average probability of DOI classification:  0.6988151844010362
SD of DOI classification:  0.44646352908745485
Number of documents w/<.2 probability of DOI classification:  234
